### 1. Preprocessing:
#### Import necessary libraries and load the data into a pandas DataFrame.

In [2]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split

In [ ]:
# load data
data = pd.read_feather('data.feather')
data.head()

#### Drop the 'ay' column and keep only the last 'is_cum' value for each triangle_id.

In [ ]:
data = data.drop("ay", axis=1)
data = data.groupby("triangle_id").last().reset_index()

#### Remove NaN values and normalize the data.

In [ ]:
data = data.fillna(0)
loss_data = data.iloc[:, 3:]
loss_data_normalized = (loss_data - loss_data.mean(axis=1).values.reshape(-1, 1)) / loss_data.std(axis=1).values.reshape(-1, 1)


#### Separate data into `X` and `y`, and reshape `X`.

In [ ]:
X = loss_data_normalized.values.reshape(-1, 1, 10, 10)
y = data["is_cum"].values

#### Split data into training, validation, and testing sets.
- 60-20-20 split

In [ ]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

## Model

#### Define the CNN architecture.

In [ ]:
class LossTriangleCNN(nn.Module):
    def __init__(self):
        super(LossTriangleCNN, self).__init__()

        # 1 input image channel, 6 output channels, 3x3 square convolution
        self.conv1 = nn.Conv2d(1, 16, 3) # 16 filters of size 3x3

        # pooling step is used to reduce the size of the output
        # in this case we are reducing the size by a factor of 2
        # 2x2 square pooling 
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # dropout layer -- randomly sets 30% of the input to 0
        self.dropout1 = nn.Dropout(0.3)

        # 6 input image channel, 16 output channels, 3x3 square convolution
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
